In [1]:
experimentsPath = "/raid/elhamod/CIFAR_HGNN/experiments/" #"/raid/elhamod/Fish/experiments/"
dataPath = "/raid/elhamod/" #"/raid/elhamod/Fish/"
experimentName = "CIFAR_scheduler_experiments2"
device = 1
detailed_reporting = False

In [ ]:
import matplotlib.pyplot as plt
import torch
import sys
import os
from sklearn.metrics import f1_score
import pandas as pd
from tqdm import tqdm
from tqdm.auto import trange
import wandb

import warnings
# warnings.filterwarnings("ignore")

try:
    import wandb
except:
    print('wandb not found')

from myhelpers import config_plots, TrialStatistics
from myhelpers.try_warning import try_running
from HGNN.train import CNN, dataLoader
from myhelpers import cifar_dataLoader
from HGNN.train.configParser import ConfigParser, getModelName, getDatasetName
config_plots.global_settings()

experimetnsFileName = "experiments.csv"
WANDB_message="wandb not working"


# For logging to server
try_running(lambda : wandb.login(), WANDB_message)

experimentPathAndName = os.path.join(experimentsPath, experimentName)
# set cuda
if device is not None:
    print("using cuda", device)
    torch.cuda.set_device(device)

else:
    print("using cpu")

# get experiment params
config_parser = ConfigParser(experimentsPath, dataPath, experimentName)

# init experiments file
experimentsFileNameAndPath = os.path.join(experimentsPath, experimetnsFileName)

paramsIterator = config_parser.getExperiments()  
number_of_experiments = sum(1 for e in paramsIterator)  
experiment_index = 0

# Loop through experiments
# with progressbar.ProgressBar(max_value=number_of_experiments) as bar:
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for experiment_params in config_parser.getExperiments():
        print(experiment_params)
        experimentHash =TrialStatistics.getTrialName(experiment_params)

        # load images 
        if experiment_params['image_path'] == 'cifar-100-python':
            datasetManager = cifar_dataLoader.datasetManager(experimentPathAndName, dataPath)
        else:
            datasetManager = dataLoader.datasetManager(experimentPathAndName, dataPath)
        datasetManager.updateParams(config_parser.fixPaths(experiment_params))
        train_loader, validation_loader, test_loader = datasetManager.getLoaders()
        architecture = {
            "fine": len(train_loader.dataset.csv_processor.getFineList()),
            "coarse" : len(train_loader.dataset.csv_processor.getCoarseList())
        }

        # Loop through n trials
        for i in trange(experiment_params["numOfTrials"], desc="trial"):
            modelName = getModelName(experiment_params, i)
            trialName = os.path.join(experimentPathAndName, modelName)
            trialHash = TrialStatistics.getTrialName(experiment_params, i)

            row_information = {
                'experimentName': experimentName,
                'modelName': modelName,
                'datasetName': getDatasetName(config_parser.fixPaths(experiment_params)),
                'experimentHash': experimentHash,
                'trialHash': trialHash
            }
            row_information = {**row_information, **experiment_params} 
            print(row_information)

            run = try_running(lambda : wandb.init(project='HGNN', group=experimentName+"-"+experimentHash, name=trialHash, config=row_information), WANDB_message) #, reinit=True

            # Train/Load model
            model = CNN.create_model(architecture, experiment_params, device=device)
#             print(model)

#             try_running(lambda : wandb.watch(model, log="all"), WANDB_message)

            if os.path.exists(CNN.getModelFile(trialName)):
                print("Model {0} found!".format(trialName))
            else:
                initModelPath = CNN.getInitModelFile(experimentPathAndName)
                if os.path.exists(initModelPath):
                    model.load_state_dict(torch.load(initModelPath))
                    print("Init Model {0} found!".format(initModelPath))
                CNN.trainModel(train_loader, validation_loader, experiment_params, model, trialName, test_loader, device=device, detailed_reporting=detailed_reporting)

            # Add to experiments file
            if os.path.exists(experimentsFileNameAndPath):
                experiments_df = pd.read_csv(experimentsFileNameAndPath)
            else:
                experiments_df = pd.DataFrame()

            record_exists = not (experiments_df[experiments_df['modelName'] == modelName][experiments_df['experimentName'] == experimentName]).empty if not experiments_df.empty else False
            if record_exists:
                experiments_df.drop(experiments_df[experiments_df['modelName'] == modelName][experiments_df['experimentName'] == experimentName].index, inplace = True) 

            experiments_df = experiments_df.append(pd.DataFrame(row_information, index=[0]), ignore_index = True)
            experiments_df.to_csv(experimentsFileNameAndPath, header=True, index=False)

            try_running(lambda : run.finish(), WANDB_message)

        bar.update()

        experiment_index = experiment_index + 1



# if __name__ == "__main__":
#     torch.multiprocessing.set_start_method('spawn')
    
#     import argparse

#     parser = argparse.ArgumentParser()
#     parser.add_argument('--cuda', required=True, type=int)
#     parser.add_argument('--experiments', required=True)
#     parser.add_argument('--data', required=True)
#     parser.add_argument('--name', required=True)
#     parser.add_argument('--detailed', required=False, action='store_true')
#     args = parser.parse_args()
#     main(experimentName=args.name, experimentsPath=args.experiments, dataPath=args.data, device=args.cuda, detailed_reporting=args.detailed)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mndhamod (use `wandb login --relogin` to force relogin)
experiment:   0%|          | 0/2 [00:00<?, ?it/s]

using cuda 1
{'image_path': 'cifar-100-python', 'suffix': None, 'img_res': 32, 'augmented': True, 'batchSize': 64, 'learning_rate': 0.001, 'numOfTrials': 1, 'fc_layers': 1, 'pretrained': True, 'epochs': 40, 'patience': -1, 'optimizer': 'adabelief', 'scheduler': 'plateau', 'weightdecay': 0.0005, 'scheduler_gamma': 0.1, 'scheduler_patience': 10, 'modelType': 'HGNN', 'two_phase_lambda': False, 'lambda': 0, 'tl_model': 'preResNet', 'link_layer': 'avgpool', 'adaptive_smoothing': True, 'adaptive_lambda': 0.1, 'adaptive_alpha': 0.5, 'tripletEnabled': False, 'regularTripletLoss': False, 'tripletSamples': 3, 'tripletSelector': 'semihard', 'tripletMargin': 0.2, 'triplet_layers_dic': 'layer2,layer3', 'phylogeny_loss': False, 'displayName': 'CIFARpre-s-HGNN', 'noSpeciesBackprop': False, 'phylogeny_loss_epsilon': 0.03}
Creating dataset...
Loading dataset...
/raid/elhamod/cifar-100-python/
Files already downloaded and verified
CIFAR normalization
Loading dataset...
/raid/elhamod/cifar-100-python/
Fi

{'experimentName': 'CIFAR_scheduler_experiments2', 'modelName': 'models/60fd4d3848461f38413024599d9deb50e23a27dba3fee2ac068c2607', 'datasetName': 'datasplits/dd10c35154ee995db5de0276a21ca1a15a77964ee53811d98f089e89', 'experimentHash': '9481160d0af0d85254d9f8b5aee42e11c800cb873f8098524be39218', 'trialHash': '60fd4d3848461f38413024599d9deb50e23a27dba3fee2ac068c2607', 'image_path': 'cifar-100-python', 'suffix': None, 'img_res': 32, 'augmented': True, 'batchSize': 64, 'learning_rate': 0.001, 'numOfTrials': 1, 'fc_layers': 1, 'pretrained': True, 'epochs': 40, 'patience': -1, 'optimizer': 'adabelief', 'scheduler': 'plateau', 'weightdecay': 0.0005, 'scheduler_gamma': 0.1, 'scheduler_patience': 10, 'modelType': 'HGNN', 'two_phase_lambda': False, 'lambda': 0, 'tl_model': 'preResNet', 'link_layer': 'avgpool', 'adaptive_smoothing': True, 'adaptive_lambda': 0.1, 'adaptive_alpha': 0.5, 'tripletEnabled': False, 'regularTripletLoss': False, 'tripletSamples': 3, 'tripletSelector': 'semihard', 'triplet

wandb: wandb version 0.10.31 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade




iteration:   0%|          | 0/40 [00:00<?, ?it/s]

...

{'experimentName': 'CIFAR_scheduler_experiments2', 'modelName': 'models/45935f01ac421813cfaef969fec9663ebbfdec021da495d50451e4e4', 'datasetName': 'datasplits/dd10c35154ee995db5de0276a21ca1a15a77964ee53811d98f089e89', 'experimentHash': '44b61f346d0316a73bcda4c9e13c40f4da6e8d08f8abdc91d8d2fe03', 'trialHash': '45935f01ac421813cfaef969fec9663ebbfdec021da495d50451e4e4', 'image_path': 'cifar-100-python', 'suffix': None, 'img_res': 32, 'augmented': True, 'batchSize': 64, 'learning_rate': 0.001, 'numOfTrials': 1, 'fc_layers': 1, 'pretrained': True, 'epochs': 40, 'patience': -1, 'optimizer': 'adabelief', 'scheduler': 'plateau', 'weightdecay': 0.0005, 'scheduler_gamma': 0.1, 'scheduler_patience': 10, 'modelType': 'DISCO', 'two_phase_lambda': False, 'lambda': 0, 'tl_model': 'preResNet', 'link_layer': 'avgpool', 'adaptive_smoothing': True, 'adaptive_lambda': 0.1, 'adaptive_alpha': 0.5, 'tripletEnabled': False, 'regularTripletLoss': False, 'tripletSamples': 3, 'tripletSelector': 'semihard', 'triple

wandb: wandb version 0.10.31 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade




iteration:   0%|          | 0/40 [00:00<?, ?it/s]

Please check your arguments if you have upgraded adabelief-pytorch from version 0.0.5.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  ---------
adabelief-pytorch=0.0.5  1e-08  False              False
>=0.1.0 (Current 0.2.0)  1e-16  True               True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended eps = 1e-8                                      Recommended eps = 1e-16
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.

Weight decoupling enabled in AdaBelief
Rectification enabled in AdaBelief
Training started...




iteration:   0%|          | 0/40 [00:26<?, ?it/s, min_val_loss=inf, train=0.00279, val=0.00261, val_loss=4.61]

iteration:   2%|▎         | 1/40 [00:26<17:05, 26.29s/it, min_val_loss=inf, train=0.00279, val=0.00261, val_loss=4.61]

iteration:   2%|▎         | 1/40 [02:31<17:05, 26.29s/it, min_val_loss=inf, train=0.747, val=0.716, val_loss=4.15]    

iteration:   5%|▌         | 2/40 [02:31<35:22, 55.85s/it, min_val_loss=inf, train=0.747, val=0.716, val_loss=4.15]

iteration:   5%|▌         | 2/40 [04:34<35:22, 55.85s/it, min_val_loss=1.4, train=0.818, val=0.773, val_loss=3.98]

iteration:   8%|▊         | 3/40 [04:34<46:58, 76.18s/it, min_val_loss=1.4, train=0.818, val=0.773, val_loss=3.98]

iteration:   8%|▊         | 3/40 [06:38<46:58, 76.18s/it, min_val_loss=1.29, train=0.842, val=0.776, val_loss=3.95]

iteration:  10%|█         | 4/40 [06:38<54:20, 90.57s/it, min_val_loss=1.29, train=0.842, val=0.776, val_loss=3.95]

iteration:  10%|█         | 4/40 [08:42<54:20, 90.57s/it, min_va

In [ ]:
print(model)